# 수집된 데이터에 대한 통계 분석
* 데이터 
> 2017년 12월 1일부터 2018년 1월 1일까지 32일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [2]:
import os
import pandas as pd 
from glob import glob
import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re
from operator import itemgetter
import html
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import chat_bot as cb
import Database_Handler as dh
from datetime import datetime
import seaborn as sns
import functools
from konlpy.tag import Mecab
from ckonlpy.tag import Twitter as ctwitter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from collections import namedtuple
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
%matplotlib inline

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
#site = 'Naver'
site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
useCollection = dh.Use_Collection(useDb, collection)

In [4]:
mecab = Mecab()
def tokenize_pos2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

In [5]:
rawdata = pd.read_csv('./data/sentiment_data/raw_data_for_sentiment.txt',header=None, encoding='utf-8')
rawdata.head()
X = list(rawdata[0])
y = np.array(list(rawdata[1]), dtype=int)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_mecab_log1 = Pipeline([
            ('vect', TfidfVectorizer(tokenizer=tokenize_pos2, ngram_range=(1,2))), 
            ('log', LogisticRegression()),
        ])

In [ ]:
%%time 
model_mecab_log1.fit(train_X, train_y)
print(classification_report(test_y, model_mecab_log1.predict(test_X)))

In [ ]:
pickle.dump(model_mecab_log1, open('./data/pre_data/classifier_log_classifier_ngram12_by_mecab.pickled','wb'))

In [6]:
model_mecab_log1 = pickle.load(open('./data/pre_data/classifier_log_classifier_ngram12_by_mecab.pickled','rb'))

In [7]:
model = doc2vec.Doc2Vec.load('./model/doc2vec_size2000_epoch20_by_mecab.model')
classifier = pickle.load(open('./data/pre_data/classifier_by_mecab_from_doc2vec_size2000_epoch20.pickled','rb'))

* 주의 
> gensim의 doc2vec 모델의 infer_vector 함수를 이용해 벡터를 생성하면, 같은 단어열(문서)에 대해서도 여러번 호출하면 각기 다른 벡터가 나온다  
> 이는 문서 벡터 추론 과정에서 랜덤하게 초기화하는 과정이 있기 때문인데, 문제는 결과로 나오는 벡터의 차이가 꽤 크다.  

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [9]:
def Cls(text, model, num):
    mecab = Mecab()
    def tokenize_pos2(doc):
        return ['/'.join(t) for t in mecab.pos(doc)]
    TaggedDocument = namedtuple('TaggedDocument', 'words tags')
    x2 = TaggedDocument(tokenize_pos2(text), 0.5)
    out = list()
    for idx in range(num):
        x = model.infer_vector(x2.words)
        cl = classifier.predict(x.reshape(1,-1))
        out.append(cl)
    out = list(itertools.chain.from_iterable(out))
    cls2 = int(round(sum(out)/len(out)))
    return cls2

In [10]:
keywordsDict = pickle.load(open('./data/pre_data/keywords_daum.pickled','rb'))
#keywordsDict = pickle.load(open('./data/pre_data/keywords_Naver.pickled','rb'))

In [13]:
i=0
outputDict  =dict()
collection = useCollection.find({'site':site})
for data in collection:
    idis = data['_id']._ObjectId__id.hex()
    text = data['title']+'. '+data['mainText']    
    commentCollection = dh.Use_Collection(useDb, 'comments')
    comments = commentCollection.find({'site':site, 'category':data['category'], 'date':data['date'], 'rank':data['rank'] })
    commEstimation = list()
    for comment in comments:
        comm = comment['comments']
        commEstimation.append(Cls(comm, model, 30))
    cls = sum(commEstimation)/ len(commEstimation)    
    outputDict[idis] = tmpdict = dict()
    tmpdict['title'] = data['title']
    tmpdict['maintextestimation'] = Cls(text, model, 50)
    tmpdict['keywordsindaum'] = data['keywords']
    tmpdict['analyzedkeywords'] = keywordsDict[idis]
    tmpdict['number_crawledcomments'] = comments.count()
    tmpdict['commentsestimation'] = cls
    tmpdict['press'] = data['press']
    tmpdict['number_of_comment'] = data['number_of_comment']
pickle.dump(outputDict, open('./outcome/outcome_daum_by_mecab.pickled','wb'))

KeyboardInterrupt: 